In [1]:
class Person:
    def _init_(self, name, weight):
        self.name = name
        self.weight = weight


    def _repr_(self):
        return f"{self.name}"


class State:
    def _init_(self, left_bank, right_bank, loc):
        self.left_bank = left_bank
        self.right_bank = right_bank
        self.loc = loc


    def _hash_(self):
        left_bank_names = "".join([p.name[0] for p in self.left_bank]) + 'l'
        right_bank_names = "".join([p.name[0] for p in self.right_bank]) + 'r'
        return hash(left_bank_names + right_bank_names + self.loc)


    def _eq_(self, other):
        return self._hash() == other.__hash_()


def deepcopy(state):
    left_bank_copy = [Person(p.name, p.weight) for p in state.left_bank]
    right_bank_copy = [Person(p.name, p.weight) for p in state.right_bank]
    return State(left_bank_copy, right_bank_copy, 'r' if state.loc == 'r' else 'l')


initial_state = State([Person("Father", 100), Person("Son", 50), Person("Son", 50)], [], 'l')
goal_state = State([], [Person("Father", 100), Person("Son", 50), Person("Son", 50)], 'r')


FLAG = [False]
SEEN = set()


stack = [(initial_state, "")]


while stack:
    state, path = stack.pop()
    
    if state in SEEN:
        continue
    
    SEEN.add(state)
    
    if state == goal_state:
        print(state._hash(), goal_state.__hash_())
        FLAG[0] = True
        print(state)
        print("Goal Reached")
        print(path)
        break
    
    if state.loc == 'l':
        for i in range(len(state.left_bank)):
            for j in range(len(state.left_bank)):
                if i == j:
                    if i < len(state.left_bank) and state.left_bank[i].weight <= 100:
                        next_state = deepcopy(state)
                        A = next_state.left_bank.pop(i)
                        next_state.right_bank.append(A)
                        next_state.loc = 'r'
                        stack.append((next_state, path + f"{A.name[0]} -> r\n"))
                    continue
                if i < len(state.left_bank) and j < len(state.left_bank) and state.left_bank[i].weight + state.left_bank[j].weight <= 100:
                    next_state = deepcopy(state)
                    A, B = next_state.left_bank[i], next_state.left_bank[j]
                    next_state.left_bank.pop(i)
                    next_state.left_bank.pop(j - 1)
                    next_state.right_bank.append(A)
                    next_state.right_bank.append(B)
                    next_state.loc = 'r'
                    stack.append((next_state, path + f"{A.name[0]}, {B.name[0]} -> r\n"))


    if state.loc == 'r':
        for i in range(len(state.right_bank)):
            for j in range(len(state.right_bank)):
                if i == j:
                    if i < len(state.right_bank) and state.right_bank[i].weight <= 100:
                        next_state = deepcopy(state)
                        A = next_state.right_bank.pop(i)
                        next_state.left_bank.append(A)
                        next_state.loc = 'l'
                        stack.append((next_state, path + f"{A.name[0]} <- l\n"))
                    continue
                if i < len(state.right_bank) and j < len(state.right_bank) and state.right_bank[i].weight + state.right_bank[j].weight <= 100:
                    next_state = deepcopy(state)
                    A, B = next_state.right_bank[i], next_state.right_bank[j]
                    next_state.right_bank.pop(i)
                    next_state.right_bank.pop(j - 1)
                    next_state.left_bank.append(A)
                    next_state.left_bank.append(B)
                    next_state.loc = 'l'
                    stack.append((next_state, path + f"{A.name[0]}, {B.name[0]} <- l\n"))

TypeError: Person() takes no arguments